**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

After importing, we will load MNIST as our data.

In [1]:
import bigdl.nano.automl as automl
automl.hpo_config.enable_hpo_tf()

from bigdl.nano.automl.tf.keras import Sequential
import bigdl.nano.automl.hpo.space as space
from bigdl.nano.tf.keras.layers import Dense, Flatten, Conv2D

from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from tensorflow import keras

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Prepare data
We will load MNIST as our data.

In [2]:
CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

img_x, img_y = x_train.shape[1], x_train.shape[2]
input_shape = (img_x, img_y, 1)
x_train = x_train.reshape(-1, img_x, img_y,1).astype("float32") / 255
x_test = x_test.reshape(-1, img_x, img_y,1).astype("float32") / 255

# Step3: Set search space
We now create our model. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [3]:
model = Sequential()

model.add(Conv2D(
    filters=space.Categorical(32, 64),
    kernel_size=space.Categorical(3, 5),
    strides=space.Categorical(1, 2),
    activation=space.Categorical("relu", "linear"),
    input_shape=input_shape))
model.add(Flatten())
model.add(Dense(CLASSES, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.0001),
    metrics=["accuracy"]
)

2022-05-19 17:56:46.391399: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-19 17:56:46.392323: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step4: Run hyperparameter search
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [4]:
%%time
model.search(
    n_trials=10,
    target_metric='val_accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-19 17:56:46,584] A new study created in memory with name: no-name-ed53ad82-d7e6-49f7-be82-3d46776bb370


Starting a new tuning
Epoch 1/24
375/375 - 8s - loss: 0.5511 - accuracy: 0.8663 - val_loss: 0.3024 - val_accuracy: 0.9124 - 8s/epoch - 22ms/step
Epoch 2/24
375/375 - 8s - loss: 0.3086 - accuracy: 0.9122 - val_loss: 0.2821 - val_accuracy: 0.9212 - 8s/epoch - 22ms/step
Epoch 3/24
375/375 - 8s - loss: 0.2908 - accuracy: 0.9173 - val_loss: 0.2753 - val_accuracy: 0.9247 - 8s/epoch - 22ms/step
Epoch 4/24
375/375 - 11s - loss: 0.2826 - accuracy: 0.9199 - val_loss: 0.2678 - val_accuracy: 0.9251 - 11s/epoch - 29ms/step
Epoch 5/24
375/375 - 10s - loss: 0.2773 - accuracy: 0.9217 - val_loss: 0.2661 - val_accuracy: 0.9280 - 10s/epoch - 27ms/step
Epoch 6/24
375/375 - 10s - loss: 0.2739 - accuracy: 0.9230 - val_loss: 0.2691 - val_accuracy: 0.9253 - 10s/epoch - 26ms/step
Epoch 7/24
375/375 - 10s - loss: 0.2707 - accuracy: 0.9244 - val_loss: 0.2641 - val_accuracy: 0.9281 - 10s/epoch - 28ms/step
Epoch 8/24
375/375 - 10s - loss: 0.2687 - accuracy: 0.9247 - val_loss: 0.2649 - val_accuracy: 0.9263 - 10s/ep

[I 2022-05-19 18:00:45,053] Trial 0 finished with value: 0.9294999837875366 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 0 with value: 0.9294999837875366.


Epoch 1/24
375/375 - 4s - loss: 1.4542 - accuracy: 0.7374 - val_loss: 0.7004 - val_accuracy: 0.8560 - 4s/epoch - 10ms/step
Epoch 2/24
375/375 - 2s - loss: 0.5216 - accuracy: 0.8677 - val_loss: 0.3706 - val_accuracy: 0.8974 - 2s/epoch - 5ms/step
Epoch 3/24
375/375 - 2s - loss: 0.3672 - accuracy: 0.8954 - val_loss: 0.3143 - val_accuracy: 0.9094 - 2s/epoch - 5ms/step
Epoch 4/24
375/375 - 3s - loss: 0.3258 - accuracy: 0.9060 - val_loss: 0.2914 - val_accuracy: 0.9173 - 3s/epoch - 8ms/step
Epoch 5/24
375/375 - 5s - loss: 0.3033 - accuracy: 0.9134 - val_loss: 0.2757 - val_accuracy: 0.9225 - 5s/epoch - 14ms/step
Epoch 6/24
375/375 - 3s - loss: 0.2866 - accuracy: 0.9181 - val_loss: 0.2645 - val_accuracy: 0.9258 - 3s/epoch - 7ms/step
Epoch 7/24
375/375 - 3s - loss: 0.2727 - accuracy: 0.9227 - val_loss: 0.2528 - val_accuracy: 0.9302 - 3s/epoch - 9ms/step
Epoch 8/24
375/375 - 3s - loss: 0.2605 - accuracy: 0.9267 - val_loss: 0.2439 - val_accuracy: 0.9324 - 3s/epoch - 7ms/step
Epoch 9/24
375/375 - 3

[I 2022-05-19 18:02:07,351] Trial 1 finished with value: 0.9629166722297668 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 1 with value: 0.9629166722297668.


...

Epoch 1/24
375/375 - 11s - loss: 1.1872 - accuracy: 0.7594 - val_loss: 0.4843 - val_accuracy: 0.8758 - 11s/epoch - 29ms/step
Epoch 2/24
375/375 - 2s - loss: 0.4179 - accuracy: 0.8830 - val_loss: 0.3340 - val_accuracy: 0.9057 - 2s/epoch - 4ms/step
Epoch 3/24
375/375 - 2s - loss: 0.3389 - accuracy: 0.9030 - val_loss: 0.2992 - val_accuracy: 0.9135 - 2s/epoch - 5ms/step
Epoch 4/24
375/375 - 2s - loss: 0.3075 - accuracy: 0.9121 - val_loss: 0.2774 - val_accuracy: 0.9223 - 2s/epoch - 5ms/step
Epoch 5/24
375/375 - 2s - loss: 0.2847 - accuracy: 0.9199 - val_loss: 0.2588 - val_accuracy: 0.9267 - 2s/epoch - 5ms/step
Epoch 6/24
375/375 - 2s - loss: 0.2650 - accuracy: 0.9252 - val_loss: 0.2430 - val_accuracy: 0.9321 - 2s/epoch - 6ms/step
Epoch 7/24
375/375 - 2s - loss: 0.2461 - accuracy: 0.9306 - val_loss: 0.2266 - val_accuracy: 0.9372 - 2s/epoch - 6ms/step
Epoch 8/24
375/375 - 3s - loss: 0.2283 - accuracy: 0.9355 - val_loss: 0.2122 - val_accuracy: 0.9422 - 3s/epoch - 7ms/step
Epoch 9/24
375/375 - 

[I 2022-05-19 18:22:21,838] Trial 8 finished with value: 0.9747499823570251 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 1, 'strides▁choice': 1}. Best is trial 5 with value: 0.9834166765213013.


Epoch 1/24
375/375 - 3s - loss: 1.2046 - accuracy: 0.7700 - val_loss: 0.5293 - val_accuracy: 0.8692 - 3s/epoch - 8ms/step
Epoch 2/24
375/375 - 3s - loss: 0.4384 - accuracy: 0.8812 - val_loss: 0.3416 - val_accuracy: 0.9027 - 3s/epoch - 7ms/step
Epoch 3/24
375/375 - 1s - loss: 0.3495 - accuracy: 0.8997 - val_loss: 0.3104 - val_accuracy: 0.9107 - 1s/epoch - 4ms/step
Epoch 4/24
375/375 - 1s - loss: 0.3241 - accuracy: 0.9072 - val_loss: 0.2989 - val_accuracy: 0.9140 - 1s/epoch - 3ms/step
Epoch 5/24
375/375 - 1s - loss: 0.3107 - accuracy: 0.9111 - val_loss: 0.2886 - val_accuracy: 0.9197 - 1s/epoch - 4ms/step
Epoch 6/24
375/375 - 1s - loss: 0.3017 - accuracy: 0.9135 - val_loss: 0.2845 - val_accuracy: 0.9211 - 1s/epoch - 4ms/step
Epoch 7/24
375/375 - 2s - loss: 0.2954 - accuracy: 0.9163 - val_loss: 0.2798 - val_accuracy: 0.9215 - 2s/epoch - 4ms/step
Epoch 8/24
375/375 - 2s - loss: 0.2908 - accuracy: 0.9172 - val_loss: 0.2766 - val_accuracy: 0.9233 - 2s/epoch - 4ms/step
Epoch 9/24
375/375 - 1s 

[I 2022-05-19 18:23:00,239] Trial 9 finished with value: 0.9284166693687439 and parameters: {'activation▁choice': 1, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 5 with value: 0.9834166765213013.


CPU times: user 1h 50min 13s, sys: 1h 27min 11s, total: 3h 17min 25s
Wall time: 26min 13s


# Step 5: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/24
375/375 [==============================] - 12s 32ms/step - loss: 0.5815 - accuracy: 0.8677 - val_loss: 0.2740 - val_accuracy: 0.9241
Epoch 2/24
375/375 [==============================] - 11s 30ms/step - loss: 0.2522 - accuracy: 0.9281 - val_loss: 0.2055 - val_accuracy: 0.9444
Epoch 3/24
375/375 [==============================] - 11s 29ms/step - loss: 0.1892 - accuracy: 0.9464 - val_loss: 0.1622 - val_accuracy: 0.9574
Epoch 4/24
375/375 [==============================] - 12s 32ms/step - loss: 0.1460 - accuracy: 0.9595 - val_loss: 0.1275 - val_accuracy: 0.9658
Epoch 5/24
375/375 [==============================] - 11s 29ms/step - loss: 0.1177 - accuracy: 0.9674 - val_loss: 0.1106 - val_accuracy: 0.9700
Epoch 6/24
375/375 [==============================] - 12s 32ms/step - loss: 0.0994 - accuracy: 0.9734 - val_loss: 0.0979 - val_accuracy: 0.9733
Epoch 7/24
375/375 [==============================] - 11s 30ms/step - loss: 0.0866 - accuracy: 0.9761 - val_loss: 0.0891 - val_accuracy:

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [6]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        1664      
                                                                 
 flatten_1 (Flatten)         (None, 36864)             0         
                                                                 
 dense_1 (Dense)             (None, 10)                368650    
                                                                 
Total params: 370,314
Trainable params: 370,314
Non-trainable params: 0
_________________________________________________________________
None
